<a href="https://colab.research.google.com/github/zoldyck13/Qwen-7B-Chat-Fine-Tuning/blob/main/Qwen_7B_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple colab to fine-tune Qwen Chat models.



## Setup(run once per session)

This code mounts your google drive, for save the fine-tuned weights.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This Download the model from the huggingface_hub.

In [2]:
!pip install huggingface_hub

from huggingface_hub import snapshot_download

snapshot_download(repo_id="Qwen/Qwen-7B-Chat",local_dir="./models/Qwen-7B-Chat", local_dir_use_symlinks=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

NOTICE: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

qwen.tiktoken: 0.00B [00:00, ?B/s]

qwen_generation_utils.py: 0.00B [00:00, ?B/s]

tokenization_qwen.py: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

'/content/models/Qwen-7B-Chat'

## Prerequisite

accelerator is installed to help the Transformers library manage devices and improve training preformance automatically.

In [3]:
!pip install -U accelerator
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

transformers is used to load, customize, and train large language model like Qwen-7B easily usign a high-level API

In [1]:
!pip uninstall -y transformers
!pip install -U transformers

Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
ERROR: Operation cancelled by user
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)


###This code performs fine-tuning on the Qwen-7B-Chat language model using the 🤗 Transformers library with the following key techniques:
 - LoRA (Low-Rank Adaptation) is applied using the peft library to reduce the
number of trainable parameters and make training more efficient.
 - The model is loaded in 4-bit using BitsAndBytesConfig to save memory and allow training on a single GPU.
 - The dataset consists of multi-turn conversations, formatted as chat between roles like [user] and [assistant].
 - A custom data collator is used to handle padding and attention masks manually.
 - The training is done using the Trainer class from Hugging Face, with gradient accumulation and mixed precision (fp16) for better performance.

This setup allows efficient fine-tuning of a large model like Qwen-7B even on limited hardware.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import BitsAndBytesConfig
from transformers import DataCollatorForLanguageModeling
from torch.nn.utils.rnn import pad_sequence
import torch
import pandas as pd
from datasets import Dataset


model_name = "./models/Qwen-7B-Chat"

tokenizer = AutoTokenizer.from_pretrained("./models/Qwen-7B-Chat", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token or "<|endoftext|>"




bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map={"": "cuda:0"}
)

model.config.pad_token_id = tokenizer.pad_token_id


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)


# dataset = load_dataset("json", data_files={"train": "./finetune_data.jsonl"}, cache_dir=None) # Removed this line
# Load the JSONL file using pandas and convert to datasets Dataset
df = pd.read_json("./finetune_data.jsonl", lines=True)
dataset = Dataset.from_pandas(df)


def preprocess_function(examples):
    all_texts = []
    for messages in examples["messages"]:
        chat = ""
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            chat += f"[{role}]: {content}\n"
        all_texts.append(chat)
    return tokenizer(all_texts, truncation=True, max_length=512)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

def custom_data_collator(features):
    input_ids = [torch.tensor(f["input_ids"]) for f in features]
    attention_mask = [torch.ones_like(ids) for ids in input_ids]
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_mask_padded,
        "labels": input_ids_padded.clone(),
    }

training_args = TrainingArguments(
    output_dir="./qwen-lora-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=custom_data_collator,
    train_dataset=tokenized_dataset, # Modified this line
    tokenizer=tokenizer,
)

trainer.train()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Map:   0%|          | 0/301 [00:00<?, ? examples/s]

/tmp/ipython-input-2-4187507103.py:92: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alifage14 (alifage14-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss


### If you're facing troubleshooting with loading the model in 4-bit quantization, try the following:

In [ ]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes

Found existing installation: bitsandbytes 0.42.0
Uninstalling bitsandbytes-0.42.0:
  Successfully uninstalled bitsandbytes-0.42.0
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
axolotl 0.12.0.dev0 requires bitsandbytes==0.46.0, but you have bitsandbytes 0.46.1 which is incompatible.
axolotl 0.12.0.dev0 requires datasets==4.0.0, but you have datasets 2.18.0 which is incompatible.
axolotl 0.12.0.dev0 requires peft==0.16.0, but you have peft 0.10.0 which is incompatible.
axolotl 0.12.0.dev0 requires tokenizers>=0.21.1, but you have tokenizers 0.19.1 which is incompatible.
axolotl 0.12.0.dev0 requires transformers==4.53.2, but you have transformers 4.40.1 which is incompatible.


## Run the model with the fine-tuned weight from ./qwen-lora-finetuned/chechpoint to see the improvements in output quality.

In [ ]:
from transformers import FlaxAutoModelForCausalLM
from peft import PeftModel

model_name = "./models/Qwen-7B-Chat"
loar_weight_dir = "./qwen-lora-finetuned/checkpoint-114"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model = PeftModel.from_pretrained(model, loar_weight_dir)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


input_text = "شلونك حبيبي"
inputs = tokenizer(input_text, return_tensors="pt")

output = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]